In [1]:
%load_ext autoreload
%autoreload 2

from krxns.config import filepaths
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact


In [ ]:
# Set this
experiment = "default_chemprop_data_titration"
copts = [elt.stem for elt in  (filepaths["spl_cv"] / experiment).glob('*')]


In [82]:
# Loss curves
@interact
def plot_loss_curves(experiment=experiment, condition=copts):
    lc = []
    cond_dir = filepaths["spl_cv"] / experiment / condition
    list()
    for fp in cond_dir.glob("split_*/metrics.csv"):

        split = pd.read_csv(fp, sep=',')
        split['split'] = int(fp.parent.stem.split('_')[-1])
        lc.append(split)

    stats = ('mean', 'std')
    lc = pd.concat(lc, axis=0).drop(columns=['step', 'train_loss_epoch'])
    lc = lc.groupby(['epoch', 'split']).agg('mean').reset_index() #.agg(['mean', 'std']) # Average over epochs
    lc = lc.groupby('epoch').agg(stats).drop(columns=['split']).reset_index()
    fig, ax = plt.subplots()
    metrics, _ = zip(*lc.columns)
    metrics = set(metrics)
    metrics.remove('epoch')
    for m in metrics:
        ax.errorbar(x=lc['epoch'], y=lc[(m, stats[0])], yerr=lc[(m, stats[1])], label=m)

    ax.set_xlabel('Epochs')
    ax.legend()
    plt.show()

interactive(children=(Text(value='default_chemprop_data_titration', description='experiment'), Dropdown(descri…